In [9]:
import os
import io
import sys
import smtplib
import locale
import warnings
import pandas as pd
import pyodbc
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email import encoders

# ================== CONFIG ==================
# SMTP
email_user = os.getenv("SMTP_USER", "gain.in")          # e.g. "alerts@sparecare.in"
email_password = os.getenv("SMTP_PASS", "")      # app password if needed
smtp_server = os.getenv("SMTP_SERVER", "smtp.gmail.com")
smtp_port = int(os.getenv("SMTP_PORT", "587"))

# SQL Server connection (pyodbc DSN)
ODBC_DSN = (
   
)

# Period order - must match SQL output
period_order = ["7 DAYS", "30 Days", "6 Months"]

DEFAULT_CC = ""

EXCEL_COL_ORDER = [
    "Selling Dealer",
    "Selling Dealer Location",
    "Buying Dealer",
    "Buying Dealer Location",
    "Order Date", "Order For",
    "Partnumber", "Part Description",
    "StockCat",
    "MRP", "Discount", "Discounted Price per qty",
    "Qty Required", "Qty Confirmed", "POQty",
    "Original Stock of Seller",
    "Rejection Date", "Rejection reason",
    "Expired Date", "Expired Reason",
    "Still_in_stock (QTY)",
    "Parts Still in Stock (in Rs)",
    # "Latest Stock of Seller",
    # "Latest Stock Value", 
    "Latest Stock Date of Seller",
   
]

# ======= YOUR FULL SQL (unchanged) =======
SQL_TEXT = r""" DECLARE @set_email  NVARCHAR(MAX),    
            @set_email2 NVARCHAR(MAX);    
    
    SET @set_email  = '';    
    SET @set_email2 = '';    
    
    WITH EmailAgg AS (
    SELECT
        lb.dealer,
        lb.DealerID,
        lb.location,
        a.locationID,
        STRING_AGG(CASE WHEN Sequence IN (3, 4) THEN a.Email ELSE NULL END, ',') AS ToEmail,
        STRING_AGG(CASE WHEN Sequence IN (1, 2, 5, 8, 9) THEN a.Email ELSE NULL END, ',') AS cc_email
    FROM LocationContactMatrixMapping AS a
    LEFT JOIN locationinfo AS lb ON a.LocationID = lb.LocationID
    WHERE Sequence IN (1,2,3,4,5,6,8,9)
      AND a.Email IS NOT NULL AND a.Email <> '' AND a.Email LIKE '%@%'
    GROUP BY lb.dealer,dealerid, lb.location, a.locationID
)
    
    --INSERT INTO seller_rejection_B    
    --(    
    --    Period,    
    --    Dealer,    
    --    dealerid,    
    --    Location,    
    --    locationid,    
    --    Partnumber,    
    --    [No of Parts],    
    --    [Rejected Order Value (in Rs)],    
    --    [Parts Still in Stock (in Rs)],    
    --    [Total Rejection],    
    --    ToEmail,    
    --    ccemail    
    --)    
   SELECT    
        Period,   
		Brand as [Brand],
		selling_dealer AS [Selling Dealer],    
		 selling_dealerid,    
		  selling_location AS [Selling Dealer Location],   
		   selling_locationid,
		   buying_dealer AS [Buying Dealer],    
		    buying_dealerid ,   
		    buying_location AS [Buying Dealer Location],  
		    buying_locationid,
			orderdate AS [Order Date],
			ORDERFOR AS [Order For],
			 '''' + Partnumber AS Partnumber,
			Partdesc AS [Part Description],
			StockCat,
			mrp AS [MRP],
			discount AS [Discount],
			Price AS [Discounted Price per qty],
			Qty_req AS [Qty Required],
			Confirm_Order AS [Qty Confirmed],
			poqty AS [POQty],
			orginal_stock_of_seller AS [Original Stock of Seller],
			reject_date AS [Rejection Date],
			REJECTREASON AS [Rejection reason],
			EXPIREDDATE AS [Expired Date],
			expiredremarks  AS [Expired Reason],
			Latest_Stock_of_Seller AS [Latest Stock of Seller],
			Latest_Stock_Value AS [Latest Stock Value],
			[Latest Stock Date of Seller],
            Min_stock_rej as [Still_in_stock (QTY)],

        SUM(Seller_Reject)         AS [No of Parts],    
        SUM(Seller_Reject_Amount)  AS [Rejected Order Value (in Rs)],    
        SUM(Still_in_Stock)        AS [Parts Still in Stock (in Rs)],    
        SUM(total_rejection)       AS [Total Rejection] ,
		 e.ToEmail,
         --'scsit.db3@sparecare.in' as ToEmail,    
        CASE WHEN cc_email IS NULL THEN concat(@set_email2,',',bf.email_new) ELSE CONCAT(cc_email, @set_email,',',bf.email_new) END AS cc_email
          --'gainer.ho@sparecare.in' as cc_email
    FROM    
    (    
 SELECT    
            '7 DAYS'       AS PERIOD,
			b.Brand,
			b.selling_dealer,    
			b. selling_dealerid,    
			b.  selling_location,   
			b.   selling_locationid,

			b.buying_dealer,    
			b. buying_dealerid,  
			b. buying_location,  
			b. buying_locationid,
			b.ORDERFOR,
            B.PARTNUMBER, 
			b.partdesc,
			b.qty_req,
			b.POQty,
            B.REQUESTDATE as Orderdate,
			b.reject_date,
			b.REJECTREASON,
			b.StockCat,
			b.Confirm_Order,
			b.[Latest Stock Date of Seller],
			b.Latest_Stock_of_Seller,
			b.Latest_Stock_Value,
			b.orginal_stock_of_seller,
			B.MRP,
			B.DISCOUNT,
			B.Price,
            B.EXPIREDDATE,  
			b.EXPIREDREMARKS,
            B.SELLER_REJECT,    
            B.SELLER_REJECT_AMOUNT,    
            B.Seller_Expired_Amount,    
            B.Pur_qty,    
            CASE    
                WHEN ISNULL(B.pur_qty, '0') = '0' THEN    
                    CASE    
                        WHEN ISNULL(B.Qty, 0) >= CASE WHEN B.reject_date IS NOT NULL THEN Seller_Reject ELSE 0 END    
                        THEN CASE WHEN B.reject_date IS NOT NULL THEN Seller_Reject ELSE 0 END    
                        ELSE ISNULL(B.Qty, 0)    
                    END    
                ELSE '0'    
            END AS Min_stock_rej,    
            CASE    
                WHEN ISNULL(B.pur_qty, '0') = '0' THEN    
                    CASE    
                        WHEN ISNULL(B.Qty, 0) >= CASE WHEN B.reject_date IS NOT NULL THEN Seller_Reject ELSE 0 END    
                        THEN CASE WHEN B.reject_date IS NOT NULL THEN Seller_Reject ELSE 0 END    
                        ELSE ISNULL(B.Qty, 0)    
                    END    
                ELSE '0'    
            END * B.PRICE AS Still_in_Stock,    
            (    
                CASE WHEN B.reject_date  IS NOT NULL THEN B.Seller_Reject_Amount  ELSE 0 END    
              + CASE WHEN B.EXPIREDDATE  IS NOT NULL THEN B.Seller_Expired_Amount ELSE 0 END    
            ) AS total_rejection    
        FROM    
        (    
            SELECT  
			e.brand as Brand,
                e.Dealer as    selling_dealer,    
                e.dealerid as   selling_dealerid,    
                e.Location as    selling_location,    
                e.LocationID as   selling_locationid, 
				
				f.Dealer as    buying_dealer,    
                f.dealerid as   buying_dealerid,    
                f.Location as   buying_location,    
                f.LocationID as buying_locationid, 
				ck.Qty as Latest_Stock_of_Seller,
				(ck.Qty * pm.landedcost) as Latest_Stock_Value,
				ck.StockDate as [Latest Stock Date of Seller],

				a.StockCat,
				a.ORDERFOR,

                a.partnumber, 
				pm.partdesc,
              a.STATUS,    
                a.REQUESTDATE,    
                a.EXPIREDDATE, 
                a.REJECTREASON,    
                a.RejectionPartyType,  
				a.MRP,
				a.DISPATCHQTY as Confirm_Order,
				A.DISCOUNT,
                a.PRICE,
				a.EXPIREDREMARKS,
				a.SELLERSTOCKQTY as orginal_stock_of_seller,
				a.QTY as qty_req,
				a.POQty,
               cs.QTY,    
                pur_qty,    
                CASE    
                    WHEN SellerFreeStock >= CASE    
                        WHEN a.status = 'dispatched'        AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'dispatchreject'    AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'REQUESTREJECT'     AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(a.QTY, 0)    
                        WHEN a.status = 'RESPONSECONFIRM'   AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'RESPONSEREQUESTSENT' AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(a.QTY, 0)    
                        ELSE 0    
                    END    
                    THEN CASE    
                        WHEN a.status = 'dispatched'        AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'dispatchreject'    AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'REQUESTREJECT'     AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(a.QTY, 0)    
                        WHEN a.status = 'RESPONSECONFIRM'   AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'RESPONSEREQUESTSENT' AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(a.QTY, 0)    
                        ELSE 0    
                    END    
                    ELSE SellerFreeStock    
                END AS Seller_Reject,    
                CASE    
                    WHEN SellerFreeStock >= CASE    
                        WHEN a.status = 'dispatched'        AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'dispatchreject'    AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'REQUESTREJECT'     AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(a.QTY, 0)    
                        WHEN a.status = 'RESPONSECONFIRM'   AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'RESPONSEREQUESTSENT' AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(a.QTY, 0)    
                        ELSE 0    
                    END    
                    THEN CASE    
                        WHEN a.status = 'dispatched'        AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'dispatchreject'    AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'REQUESTREJECT'     AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(a.QTY, 0)    
                        WHEN a.status = 'RESPONSECONFIRM'   AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'RESPONSEREQUESTSENT' AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(a.QTY, 0)    
                        ELSE 0    
                    END    
                    ELSE SellerFreeStock    
                END * price AS Seller_Reject_Amount,    
                CASE    
                    WHEN orderstatus = 'reject'    
                         AND (    
                                (requestdate IS NOT NULL AND requestacceptdate IS NOT NULL    
                                 AND responseconfirmdate IS NULL    
            AND dispatchdate IS NULL    
                                 AND FURTHERDETAILSDATE IS NULL    
                                 AND FURTHERDETAILSACCEPTEDDATE IS NULL)    
                             OR (requestdate IS NOT NULL AND requestacceptdate IS NOT NULL    
             AND FURTHERDETAILSDATE IS NOT NULL AND FURTHERDETAILSACCEPTEDDATE IS NOT NULL    
                                 AND responseconfirmdate IS NULL    
                                 AND dispatchdate IS NULL)    
                             )    
                    THEN REQUESTACCEPTDATE    
                    WHEN orderstatus = 'reject'    
                         AND (    
                                (requestdate IS NOT NULL AND requestacceptdate IS NOT NULL    
                                 AND responseconfirmdate IS NOT NULL    
                                 AND dispatchdate IS NOT NULL    
                                 AND FURTHERDETAILSDATE IS NULL    
                                 AND FURTHERDETAILSACCEPTEDDATE IS NULL)    
                             OR (requestdate IS NOT NULL AND requestacceptdate IS NOT NULL    
                                 AND FURTHERDETAILSDATE IS NOT NULL AND FURTHERDETAILSACCEPTEDDATE IS NOT NULL    
                                 AND responseconfirmdate IS NOT NULL    
                                 AND dispatchdate IS NOT NULL)    
                             )    
                    THEN DISPATCHDATE   
                    ELSE ''    
                END AS reject_date,    
                CASE    
                    WHEN a.STATUS = 'dispatched'       AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                    WHEN a.status = 'REQUESTreject'    AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(a.qty, 0)    
                    WHEN a.status = 'REQUESTsent'      AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(a.qty, 0)    
                    WHEN a.status = 'RESPONSECONFIRM'  AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(POqty, 0)    
                    WHEN a.status = 'RESPONSEreject'   AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(dispatchqty, 0)    
                    WHEN a.status = 'RESPONSEREQUESTSENT' AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(a.qty, 0)    
                    ELSE 0    
                END AS Seller_Expired,    
                CASE    
                    WHEN a.STATUS = 'dispatched'       AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                    WHEN a.status = 'REQUESTreject'    AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(a.qty, 0)    
                    WHEN a.status = 'REQUESTsent'      AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(a.qty, 0)    
                    WHEN a.status = 'RESPONSECONFIRM'  AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(POqty, 0)    
                    WHEN a.status = 'RESPONSEreject'   AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(dispatchqty, 0)    
                    WHEN a.status = 'RESPONSEREQUESTSENT' AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(a.qty, 0)    
                    ELSE 0    
                END * Price AS Seller_Expired_Amount    
            FROM SH_PartTransaction (NOLOCK) AS a  
			left join (select a.locationid,a.stockdate,b.partnumber,b.qty
			from CurrentStock1 as a
			left join CurrentStock2 as b
			on a.tCode = b.StockCode
			) as ck on ck.LocationID = a.SELLERLOCATION and a.PARTNUMBER = ck.PartNumber
            LEFT JOIN z_scope..locationinfo (NOLOCK) AS e    
                ON a.sellerlocation = e.locationid  
			Left Join z_scope..locationinfo (NOLOCK) AS f
			    ON a.BUYERLOCATION = f.LocationID
			left join part_master as pm on e.brandid = pm.brandid and a.partnumber = pm.partnumber
            LEFT JOIN    
            (    
                SELECT locationid, partnumber, SUM(qty) AS pur_qty    
                FROM    
                (    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_vor_9]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'cp'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_vor_11]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'cp'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_vor_17]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'cp'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_vor_18]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'cp'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_vor_28]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'cp'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_vor_32]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'cp'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_vor_24]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'cp'    
                      AND qty > 0    
    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_old_9]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'p'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_old_11]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'p'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_old_17]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                     >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'p'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_old_18]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'p'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_old_28]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'p'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_old_32]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'p'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_old_24]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'p'    
                      AND qty > 0    
                ) AS a    
                GROUP BY locationid, partnumber    
            ) AS PUR    
                ON a.SELLERLOCATION = PUR.Locationid    
               AND CONCAT('''', a.PARTNUMBER) = PUR.partnumber    
            LEFT JOIN    
            (    
                SELECT a.locationid, b.PartNumber, b.Qty, StockDate    
                FROM z_scope..currentstock1 (NOLOCK) AS a    
                JOIN z_scope..currentstock2 (NOLOCK) AS b    
                  ON a.tCode = b.StockCode    
            ) AS cs    
              ON a.SELLERLOCATION = cs.LocationID    
             AND a.PARTNUMBER    = cs.PartNumber    
        ) AS B    
        WHERE    
            CAST(B.RequestDate AS DATE) >= CAST(DATEADD(DAY, -7, GETDATE()) AS DATE)    
            AND Seller_Reject_Amount >= 5000    
            AND (B.REJECTREASON <> 'Auto Rejected' OR B.REJECTREASON IS NULL)    
            AND (B.RejectionPartyType = 'Seller' OR B.RejectionPartyType IS NULL)  
    
        UNION ALL    
    
        -------------------------------------------------------------------------    
        -- Last 30 Days  
        -------------------------------------------------------------------------    
        SELECT    
            '30 Days'    AS PERIOD,  
			b.Brand,
			b.selling_dealer,    
			b. selling_dealerid,    
			b.  selling_location,   
			b.   selling_locationid,

			b.buying_dealer,    
			b. buying_dealerid,  
			b. buying_location,  
			b. buying_locationid,
			b.ORDERFOR,
            B.PARTNUMBER, 
			b.partdesc,
			b.qty_req,
			b.POQty,
            B.REQUESTDATE as Orderdate,
			b.reject_date,
			b.REJECTREASON,
			b.StockCat,
			b.Confirm_Order,
			b.[Latest Stock Date of Seller],
			b.Latest_Stock_of_Seller,
			b.Latest_Stock_Value,
			b.orginal_stock_of_seller,
			B.MRP,
			B.DISCOUNT,
			B.Price,
            B.EXPIREDDATE,  
			b.EXPIREDREMARKS,
            B.SELLER_REJECT,    
            B.SELLER_REJECT_AMOUNT,    
            B.Seller_Expired_Amount,    
            B.Pur_qty,    
            CASE    
                WHEN ISNULL(B.pur_qty, '0') = '0' THEN    
                    CASE    
                        WHEN ISNULL(B.Qty, 0) >= CASE WHEN B.reject_date IS NOT NULL THEN Seller_Reject ELSE 0 END    
                        THEN CASE WHEN B.reject_date IS NOT NULL THEN Seller_Reject ELSE 0 END    
                        ELSE ISNULL(B.Qty, 0)    
                    END    
                ELSE '0'    
            END AS Min_stock_rej,    
            CASE    
                WHEN ISNULL(B.pur_qty, '0') = '0' THEN    
                    CASE    
                        WHEN ISNULL(B.Qty, 0) >= CASE WHEN B.reject_date IS NOT NULL THEN Seller_Reject ELSE 0 END    
                        THEN CASE WHEN B.reject_date IS NOT NULL THEN Seller_Reject ELSE 0 END    
                        ELSE ISNULL(B.Qty, 0)    
                    END    
                ELSE '0'    
            END * B.PRICE AS Still_in_Stock,    
            (    
                CASE WHEN B.reject_date  IS NOT NULL THEN B.Seller_Reject_Amount  ELSE 0 END    
              + CASE WHEN B.EXPIREDDATE  IS NOT NULL THEN B.Seller_Expired_Amount ELSE 0 END    
            ) AS total_rejection    
        FROM    
        (    
            SELECT    
			e.brand  as Brand,
                e.Dealer as    selling_dealer,    
                e.dealerid as   selling_dealerid,    
                e.Location as    selling_location,    
                e.LocationID as   selling_locationid, 
				
				f.Dealer as    buying_dealer,    
                f.dealerid as   buying_dealerid,    
                f.Location as   buying_location,    
                f.LocationID as buying_locationid, 
				ck.Qty as Latest_Stock_of_Seller,
				(ck.Qty * pm.landedcost) as Latest_Stock_Value,
				ck.StockDate as [Latest Stock Date of Seller],

				a.StockCat,
				a.ORDERFOR,

                a.partnumber, 
				pm.partdesc,
              a.STATUS,    
                a.REQUESTDATE,    
                a.EXPIREDDATE, 
                a.REJECTREASON,    
                a.RejectionPartyType,  
				a.MRP,
				a.DISPATCHQTY as Confirm_Order,
				A.DISCOUNT,
                a.PRICE,
				a.EXPIREDREMARKS,
				a.SELLERSTOCKQTY as orginal_stock_of_seller,
				a.QTY as qty_req,
				a.POQty,
               cs.QTY,    
                pur_qty,    
                CASE    
                    WHEN SellerFreeStock >= CASE    
                        WHEN a.status = 'dispatched'        AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'dispatchreject'    AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'REQUESTREJECT'     AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(a.QTY, 0)    
                        WHEN a.status = 'RESPONSECONFIRM'   AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'RESPONSEREQUESTSENT' AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(a.QTY, 0)    
                        ELSE 0    
                    END    
                    THEN CASE    
                        WHEN a.status = 'dispatched'        AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'dispatchreject'    AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'REQUESTREJECT'     AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(a.QTY, 0)    
                        WHEN a.status = 'RESPONSECONFIRM'   AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'RESPONSEREQUESTSENT' AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(a.QTY, 0)    
                        ELSE 0    
                    END    
                    ELSE SellerFreeStock    
                END AS Seller_Reject,    
                CASE    
                    WHEN SellerFreeStock >= CASE    
                        WHEN a.status = 'dispatched'        AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'dispatchreject'    AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'REQUESTREJECT'     AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(a.QTY, 0)    
                        WHEN a.status = 'RESPONSECONFIRM'   AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'RESPONSEREQUESTSENT' AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(a.QTY, 0)    
                        ELSE 0    
                    END    
                    THEN CASE    
                        WHEN a.status = 'dispatched'        AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'dispatchreject'    AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'REQUESTREJECT'     AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(a.QTY, 0)    
                        WHEN a.status = 'RESPONSECONFIRM'   AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'RESPONSEREQUESTSENT' AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(a.QTY, 0)    
                        ELSE 0    
                    END    
                    ELSE SellerFreeStock    
                END * price AS Seller_Reject_Amount,    
                CASE    
                    WHEN orderstatus = 'reject'    
                         AND (    
                                (requestdate IS NOT NULL AND requestacceptdate IS NOT NULL    
                                 AND responseconfirmdate IS NULL    
            AND dispatchdate IS NULL    
                                 AND FURTHERDETAILSDATE IS NULL    
                                 AND FURTHERDETAILSACCEPTEDDATE IS NULL)    
                             OR (requestdate IS NOT NULL AND requestacceptdate IS NOT NULL    
             AND FURTHERDETAILSDATE IS NOT NULL AND FURTHERDETAILSACCEPTEDDATE IS NOT NULL    
                                 AND responseconfirmdate IS NULL    
                                 AND dispatchdate IS NULL)    
                             )    
                    THEN REQUESTACCEPTDATE    
                    WHEN orderstatus = 'reject'    
                         AND (    
                                (requestdate IS NOT NULL AND requestacceptdate IS NOT NULL    
                                 AND responseconfirmdate IS NOT NULL    
                                 AND dispatchdate IS NOT NULL    
                                 AND FURTHERDETAILSDATE IS NULL    
                                 AND FURTHERDETAILSACCEPTEDDATE IS NULL)    
                             OR (requestdate IS NOT NULL AND requestacceptdate IS NOT NULL    
                                 AND FURTHERDETAILSDATE IS NOT NULL AND FURTHERDETAILSACCEPTEDDATE IS NOT NULL    
                                 AND responseconfirmdate IS NOT NULL    
                                 AND dispatchdate IS NOT NULL)    
                             )    
                    THEN DISPATCHDATE   
                    ELSE ''    
                END AS reject_date,    
                CASE    
                    WHEN a.STATUS = 'dispatched'       AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                    WHEN a.status = 'REQUESTreject'    AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(a.qty, 0)    
                    WHEN a.status = 'REQUESTsent'      AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(a.qty, 0)    
                    WHEN a.status = 'RESPONSECONFIRM'  AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(POqty, 0)    
                    WHEN a.status = 'RESPONSEreject'   AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(dispatchqty, 0)    
                    WHEN a.status = 'RESPONSEREQUESTSENT' AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(a.qty, 0)    
                    ELSE 0    
                END AS Seller_Expired,    
                CASE    
                    WHEN a.STATUS = 'dispatched'       AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                    WHEN a.status = 'REQUESTreject'    AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(a.qty, 0)    
                    WHEN a.status = 'REQUESTsent'      AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(a.qty, 0)    
                    WHEN a.status = 'RESPONSECONFIRM'  AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(POqty, 0)    
                    WHEN a.status = 'RESPONSEreject'   AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(dispatchqty, 0)    
                    WHEN a.status = 'RESPONSEREQUESTSENT' AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(a.qty, 0)    
                    ELSE 0    
                END * Price AS Seller_Expired_Amount    
            FROM SH_PartTransaction (NOLOCK) AS a  
			left join (select a.locationid,a.stockdate,b.partnumber,b.qty
			from CurrentStock1 as a
			left join CurrentStock2 as b
			on a.tCode = b.StockCode
			) as ck on ck.LocationID = a.SELLERLOCATION and a.PARTNUMBER = ck.PartNumber
            LEFT JOIN z_scope..locationinfo (NOLOCK) AS e    
                ON a.sellerlocation = e.locationid  
			Left Join z_scope..locationinfo (NOLOCK) AS f
			    ON a.BUYERLOCATION = f.LocationID
			left join part_master as pm on e.brandid = pm.brandid and a.partnumber = pm.partnumber
            LEFT JOIN    
            (    
                SELECT locationid, partnumber, SUM(qty) AS pur_qty    
                FROM    
                (    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_vor_9]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'cp'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_vor_11]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'cp'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_vor_17]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'cp'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_vor_18]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'cp'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_vor_28]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'cp'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_vor_32]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'cp'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_vor_24]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'cp'    
                      AND qty > 0    
    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_old_9]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'p'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_old_11]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'p'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_old_17]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                     >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'p'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_old_18]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'p'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_old_28]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'p'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_old_32]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'p'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_old_24]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'p'    
                      AND qty > 0    
                ) AS a    
                GROUP BY locationid, partnumber    
            ) AS PUR    
                ON a.SELLERLOCATION = PUR.Locationid    
               AND CONCAT('''', a.PARTNUMBER) = PUR.partnumber    
            LEFT JOIN    
            (    
                SELECT a.locationid, b.PartNumber, b.Qty, StockDate    
                FROM z_scope..currentstock1 (NOLOCK) AS a    
                JOIN z_scope..currentstock2 (NOLOCK) AS b    
                  ON a.tCode = b.StockCode    
            ) AS cs    
              ON a.SELLERLOCATION = cs.LocationID    
             AND a.PARTNUMBER    = cs.PartNumber    
        ) AS B    
        WHERE    
              CAST(B.RequestDate AS DATE) >= CAST(DATEADD(DAY, -30, GETDATE()) AS DATE)
            AND Seller_Reject_Amount >= 5000    
            AND (B.REJECTREASON <> 'Auto Rejected' OR B.REJECTREASON IS NULL)    
            AND (B.RejectionPartyType = 'Seller' OR B.RejectionPartyType IS NULL)  
    
        UNION ALL    
    
        -------------------------------------------------------------------------    
        -- LAST 6 MONTHS (YTD WINDOWED TO LAST 6 MONTHS)    
        -------------------------------------------------------------------------    
       SELECT    
            '6 Months'   AS PERIOD, 
			b.Brand,
			b.selling_dealer,    
			b. selling_dealerid,    
			b.  selling_location,   
			b.   selling_locationid,

			b.buying_dealer,    
			b. buying_dealerid,  
			b. buying_location,  
			b. buying_locationid,
			b.ORDERFOR,
            B.PARTNUMBER, 
			b.partdesc,
			b.qty_req,
			b.POQty,
            B.REQUESTDATE as Orderdate,
			b.reject_date,
			b.REJECTREASON,
			b.StockCat,
			b.Confirm_Order,
			b.[Latest Stock Date of Seller],
			b.Latest_Stock_of_Seller,
			b.Latest_Stock_Value,
			b.orginal_stock_of_seller,
			B.MRP,
			B.DISCOUNT,
			B.Price,
            B.EXPIREDDATE,  
			b.EXPIREDREMARKS,
            B.SELLER_REJECT,    
            B.SELLER_REJECT_AMOUNT,    
            B.Seller_Expired_Amount,    
            B.Pur_qty,    
            CASE    
                WHEN ISNULL(B.pur_qty, '0') = '0' THEN    
                    CASE    
                        WHEN ISNULL(B.Qty, 0) >= CASE WHEN B.reject_date IS NOT NULL THEN Seller_Reject ELSE 0 END    
                        THEN CASE WHEN B.reject_date IS NOT NULL THEN Seller_Reject ELSE 0 END    
                        ELSE ISNULL(B.Qty, 0)    
                    END    
                ELSE '0'    
            END AS Min_stock_rej,    
            CASE    
                WHEN ISNULL(B.pur_qty, '0') = '0' THEN    
                    CASE    
                        WHEN ISNULL(B.Qty, 0) >= CASE WHEN B.reject_date IS NOT NULL THEN Seller_Reject ELSE 0 END    
                        THEN CASE WHEN B.reject_date IS NOT NULL THEN Seller_Reject ELSE 0 END    
                        ELSE ISNULL(B.Qty, 0)    
                    END    
                ELSE '0'    
            END * B.PRICE AS Still_in_Stock,    
            (    
                CASE WHEN B.reject_date  IS NOT NULL THEN B.Seller_Reject_Amount  ELSE 0 END    
              + CASE WHEN B.EXPIREDDATE  IS NOT NULL THEN B.Seller_Expired_Amount ELSE 0 END    
            ) AS total_rejection    
        FROM    
        (    
            SELECT   
			e.Brand as Brand,
                e.Dealer as    selling_dealer,    
                e.dealerid as   selling_dealerid,    
                e.Location as    selling_location,    
                e.LocationID as   selling_locationid, 
				
				f.Dealer as    buying_dealer,    
                f.dealerid as   buying_dealerid,    
                f.Location as   buying_location,    
                f.LocationID as buying_locationid, 
				ck.Qty as Latest_Stock_of_Seller,
				(ck.Qty * pm.landedcost) as Latest_Stock_Value,
				ck.StockDate as [Latest Stock Date of Seller],

				a.StockCat,
				a.ORDERFOR,

                a.partnumber, 
				pm.partdesc,
              a.STATUS,    
                a.REQUESTDATE,    
                a.EXPIREDDATE, 
                a.REJECTREASON,    
                a.RejectionPartyType,  
				a.MRP,
				a.DISPATCHQTY as Confirm_Order,
				A.DISCOUNT,
                a.PRICE,
				a.EXPIREDREMARKS,
				a.SELLERSTOCKQTY as orginal_stock_of_seller,
				a.QTY as qty_req,
				a.POQty,
               cs.QTY,    
                pur_qty,    
                CASE    
                    WHEN SellerFreeStock >= CASE    
                        WHEN a.status = 'dispatched'        AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'dispatchreject'    AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'REQUESTREJECT'     AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(a.QTY, 0)    
                        WHEN a.status = 'RESPONSECONFIRM'   AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'RESPONSEREQUESTSENT' AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(a.QTY, 0)    
                        ELSE 0    
                    END    
                    THEN CASE    
                        WHEN a.status = 'dispatched'        AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'dispatchreject'    AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'REQUESTREJECT'     AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(a.QTY, 0)    
                        WHEN a.status = 'RESPONSECONFIRM'   AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'RESPONSEREQUESTSENT' AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(a.QTY, 0)    
                        ELSE 0    
                    END    
                    ELSE SellerFreeStock    
                END AS Seller_Reject,    
                CASE    
                    WHEN SellerFreeStock >= CASE    
                        WHEN a.status = 'dispatched'        AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'dispatchreject'    AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'REQUESTREJECT'     AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(a.QTY, 0)    
                        WHEN a.status = 'RESPONSECONFIRM'   AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'RESPONSEREQUESTSENT' AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(a.QTY, 0)    
                        ELSE 0    
                    END    
                    THEN CASE    
                        WHEN a.status = 'dispatched'        AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'dispatchreject'    AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'REQUESTREJECT'     AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(a.QTY, 0)    
                        WHEN a.status = 'RESPONSECONFIRM'   AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                        WHEN a.status = 'RESPONSEREQUESTSENT' AND ORDERSTATUS = 'reject' AND requestdate IS NOT NULL THEN ISNULL(a.QTY, 0)    
                        ELSE 0    
                    END    
                    ELSE SellerFreeStock    
                END * price AS Seller_Reject_Amount,    
                CASE    
                    WHEN orderstatus = 'reject'    
                         AND (    
                                (requestdate IS NOT NULL AND requestacceptdate IS NOT NULL    
                                 AND responseconfirmdate IS NULL    
            AND dispatchdate IS NULL    
                                 AND FURTHERDETAILSDATE IS NULL    
                                 AND FURTHERDETAILSACCEPTEDDATE IS NULL)    
                             OR (requestdate IS NOT NULL AND requestacceptdate IS NOT NULL    
             AND FURTHERDETAILSDATE IS NOT NULL AND FURTHERDETAILSACCEPTEDDATE IS NOT NULL    
                                 AND responseconfirmdate IS NULL    
                                 AND dispatchdate IS NULL)    
                             )    
                    THEN REQUESTACCEPTDATE    
                    WHEN orderstatus = 'reject'    
                         AND (    
                                (requestdate IS NOT NULL AND requestacceptdate IS NOT NULL    
                                 AND responseconfirmdate IS NOT NULL    
                                 AND dispatchdate IS NOT NULL    
                                 AND FURTHERDETAILSDATE IS NULL    
                                 AND FURTHERDETAILSACCEPTEDDATE IS NULL)    
                             OR (requestdate IS NOT NULL AND requestacceptdate IS NOT NULL    
                                 AND FURTHERDETAILSDATE IS NOT NULL AND FURTHERDETAILSACCEPTEDDATE IS NOT NULL    
                                 AND responseconfirmdate IS NOT NULL    
                                 AND dispatchdate IS NOT NULL)    
                             )    
                    THEN DISPATCHDATE   
                    ELSE ''    
                END AS reject_date,    
                CASE    
                    WHEN a.STATUS = 'dispatched'       AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                    WHEN a.status = 'REQUESTreject'    AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(a.qty, 0)    
                    WHEN a.status = 'REQUESTsent'      AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(a.qty, 0)    
                    WHEN a.status = 'RESPONSECONFIRM'  AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(POqty, 0)    
                    WHEN a.status = 'RESPONSEreject'   AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(dispatchqty, 0)    
                    WHEN a.status = 'RESPONSEREQUESTSENT' AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(a.qty, 0)    
                    ELSE 0    
                END AS Seller_Expired,    
                CASE    
                    WHEN a.STATUS = 'dispatched'       AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(poqty, 0)    
                    WHEN a.status = 'REQUESTreject'    AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(a.qty, 0)    
                    WHEN a.status = 'REQUESTsent'      AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(a.qty, 0)    
                    WHEN a.status = 'RESPONSECONFIRM'  AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(POqty, 0)    
                    WHEN a.status = 'RESPONSEreject'   AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(dispatchqty, 0)    
                    WHEN a.status = 'RESPONSEREQUESTSENT' AND ORDERSTATUS = 'expired' AND requestdate IS NOT NULL THEN ISNULL(a.qty, 0)    
                    ELSE 0    
                END * Price AS Seller_Expired_Amount    
            FROM SH_PartTransaction (NOLOCK) AS a  
			left join (select a.locationid,a.stockdate,b.partnumber,b.qty
			from CurrentStock1 as a
			left join CurrentStock2 as b
			on a.tCode = b.StockCode
			) as ck on ck.LocationID = a.SELLERLOCATION and a.PARTNUMBER = ck.PartNumber
            LEFT JOIN z_scope..locationinfo (NOLOCK) AS e    
                ON a.sellerlocation = e.locationid  
			Left Join z_scope..locationinfo (NOLOCK) AS f
			    ON a.BUYERLOCATION = f.LocationID
			left join part_master as pm on e.brandid = pm.brandid and a.partnumber = pm.partnumber
            LEFT JOIN    
            (    
                SELECT locationid, partnumber, SUM(qty) AS pur_qty    
                FROM    
                (    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_vor_9]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'cp'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_vor_11]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'cp'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_vor_17]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'cp'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_vor_18]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'cp'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_vor_28]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'cp'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_vor_32]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'cp'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_vor_24]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'cp'    
                      AND qty > 0    
    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_old_9]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'p'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_old_11]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'p'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_old_17]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                     >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'p'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_old_18]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'p'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_old_28]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'p'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_old_32]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'p'    
                      AND qty > 0    
                    UNION ALL    
                    SELECT locationid, CONCAT('''', partnumber) AS partnumber, qty    
                    FROM uad_bi..[UAD_DRD_old_24]    
                    WHERE CAST(CONCAT('01-', LEFT(Data_Type, 3), '-', SUBSTRING(data_Type, 5, 2)) AS DATE)    
                          >= CAST(DATEADD(MONTH, -6, DATEADD(DAY, -DAY(GETDATE() - 1), GETDATE())) AS DATE)    
                      AND RIGHT(data_type, 2) = 'p'    
                      AND qty > 0    
                ) AS a    
                GROUP BY locationid, partnumber    
            ) AS PUR    
                ON a.SELLERLOCATION = PUR.Locationid    
               AND CONCAT('''', a.PARTNUMBER) = PUR.partnumber    
            LEFT JOIN    
            (    
                SELECT a.locationid, b.PartNumber, b.Qty, StockDate    
                FROM z_scope..currentstock1 (NOLOCK) AS a    
                JOIN z_scope..currentstock2 (NOLOCK) AS b    
                  ON a.tCode = b.StockCode    
            ) AS cs    
              ON a.SELLERLOCATION = cs.LocationID    
             AND a.PARTNUMBER    = cs.PartNumber    
        ) AS B    
        WHERE    
          B.RequestDate >= DATEADD(MONTH, -6, GETDATE())     
            AND Seller_Reject_Amount >= 5000    
            AND (B.REJECTREASON <> 'Auto Rejected' OR B.REJECTREASON IS NULL)    
            AND (B.RejectionPartyType = 'Seller' OR B.RejectionPartyType IS NULL)    
    ) AS s    
     LEFT JOIN EmailAgg AS e    
        ON e.LocationID = s.selling_locationid
		left join
(
select * from(
select a.bigid,a.bdmemail,a.zbdmemail,case when a.BDMCode is not null and a.ZBDMCode is not null and a.BDMCode <> a.ZBDMCode then CONCAT(a.BdmEmail,',',a.Zbdmemail)
when a.BDMCode is not null and a.ZBDMCode is null then a.BdmEmail
when a.BDMCode = a.ZBDMCode then a.Zbdmemail
end as email_new
from(
SELECT  
a.BDMCode,
c.oemail as BdmEmail,
    a.ZBDMCode,
	a.bigid,
    CASE 
        WHEN a.ZBDMCode = '135170' THEN 'arnab@sparecare.in'
        WHEN a.ZBDMCode = '147765' THEN 'rahul.aggarwal@sparecare.in'
        WHEN a.ZBDMCode = '10877'  THEN 'rahul.gandhi@sparecare.in'
    END AS Zbdmemail,
	b.oemail as admin_mail
FROM dealer_master AS a

LEFT JOIN AdminMaster_GEN AS b 
    ON a.ZBDMCode = b.bintId_Pk
left join AdminMaster_GEN as c
on a.BDMCode = c.bintId_Pk
) as a) as b
)
as bf on e.DealerID = bf.bigid
		where stockcat = 'N'
        and discount >=20
		and selling_dealer <> buying_dealer
			group by 
			Period,  
			Brand,
			selling_dealer,    
			 selling_dealerid,    
			  selling_location,   
			   selling_locationid,
			   buying_dealer,    
			    buying_dealerid,  
			    buying_location,  
			    buying_locationid,
				orderdate,
				Partdesc,
				mrp,
				discount,
				Price,
				Qty_req,
				POQty,
				EXPIREDREMARKS,
				ORDERFOR,
				Confirm_Order,
				orginal_stock_of_seller,
				PARTNUMBER,
				Latest_Stock_of_Seller,
			Latest_Stock_Value,
            Min_stock_rej,
			[Latest Stock Date of Seller],
			reject_date ,
			REJECTREASON ,
			EXPIREDDATE ,
			expiredremarks,
			StockCat,
			 e.ToEmail,    
            E.cc_email,
			bf.email_new	    
    ORDER BY    
         PERIOD,[Selling Dealer];
"""

# ================== INR FORMAT ==================
def _indian_format_fallback(n: int) -> str:
    s = str(int(n))
    if len(s) <= 3:
        return s
    head, tail = s[:-3], s[-3:]
    parts = []
    while len(head) > 2:
        parts.append(head[-2:])
        head = head[:-2]
    if head:
        parts.append(head)
    return ",".join(reversed(parts)) + "," + tail

def inr_format(x):
    try:
        n = int(round(float(0 if x is None or x == "" else x)))
    except Exception:
        n = 0
    try:
        for loc in ("en_IN.utf8", "en_IN", "en_IN.UTF-8"):
            try:
                locale.setlocale(locale.LC_ALL, loc)
                break
            except Exception:
                continue
        return locale.format_string("%d", n, grouping=True)
    except Exception:
        return _indian_format_fallback(n)

# ================== MAIL BODY ==================
def build_html_body(dealer, location, group_summary, last6_value):
    styles = """
        table { border-collapse: collapse; }
        th, td { border:1px solid #999; padding:6px 10px; text-align:center; }
        th { background:#E0E0E0; }
        p { margin:8px 0; }
    """
    rows_html = "".join(
        f"""
        <tr>
            <td>{row['Period']}</td>
            <td>{int(row['No of Parts'])}</td>
            <td>₹{inr_format(row['Rejected Order Value (in Rs)'])}</td>
            <td>₹{inr_format(row['Parts Still in Stock (in Rs)'])}</td>
        </tr>
        """
        for _, row in group_summary.iterrows()
    )
    last6_inr = f"₹{inr_format(last6_value)}"
    return f"""
    <html>
    <head><meta charset="utf-8"><style>{styles}</style></head>
    <body>
        <p>Dear Sir,</p>
        <p>In reference to Liquidation of Non-Moving Parts of <b>{dealer} - {location}</b>.</p>
        <p>Team Gainer has arranged orders from Co-Dealer for your Non-Moving Parts (as per parts listed on Portal).<br>
        But unfortunately, <b>Orders worth {last6_inr} /-</b> is REJECTED by your dealership.</p>
        <p>And some of the parts are still available in stock at your dealership.</p>
        <p><b>Order Rejection Status</b></p>
        <table>
            <tr>
                <th>Period</th>
                <th>No of Parts</th>
                <th>Rejected Order Value (in Rs)</th>
                <th>Parts Still in Stock (in Rs)</th>
            </tr>
            {rows_html}
        </table>
        <p># Part list attached in mail</p>
        <p>Order Rejection will <b><u>RESTRICT LIQUIDATION of NON-MOVING (NM) Parts</u></b> and will affect your SELLER RANKING.</p>
        <p>In case there is issue in Non-Moving Parts listing, please check &amp; revert.</p>
        <p>Thanks &amp; Regards<br>Team Gainer</p>
    </body>
    </html>
    """

# ================== HELPERS ==================
def split_email_list(value):
    if not isinstance(value, str) or not value.strip():
        return []
    # guard against accidental floats coming as "123.0"
    value = str(value)
    return [e.strip() for e in value.split(",") if "@" in e]

# ================== MAIN ==================
def main():
    conn = None
    try:
        warnings.filterwarnings("ignore", message="pandas only supports SQLAlchemy")

        # Connect & read
        conn = pyodbc.connect(ODBC_DSN, autocommit=True)
        df = pd.read_sql_query(SQL_TEXT, conn)

        if df.empty:
            print("No data to process.")
            return

        # Normalize column names we use in Python
        rename_map = {"cc_email": "ccemail", "ToEmail": "toemail"}
        df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns}, inplace=True)

        # Ensure email cols exist, are strings (avoid float .split error)
        for col in ("toemail", "ccemail"):
            if col not in df.columns:
                df[col] = ""
            df[col] = df[col].fillna("").astype(str)
            df.loc[df[col].str.strip().str.lower().isin(["nan", "none", "null"]), col] = ""

        # Ensure required grouping columns exist
        required_cols = [
            "Selling Dealer", "selling_dealerid",
            "Selling Dealer Location", "selling_locationid",
            "toemail", "ccemail", "Period",
            "No of Parts", "Rejected Order Value (in Rs)",
            "Parts Still in Stock (in Rs)", "Total Rejection",
            "Partnumber"
        ]
        for c in required_cols:
            if c not in df.columns:
                raise KeyError(f"Expected column '{c}' not found from SQL.")

        # Period ordering
        df["Period"] = pd.Categorical(df["Period"], categories=period_order, ordered=True)

        # Group by dealer+location+email set
        group_cols = [
            "Selling Dealer", "selling_dealerid",
            "Selling Dealer Location", "selling_locationid",
            "toemail", "ccemail"
        ]
        grouped = df.groupby(group_cols, dropna=False)

        for keys, chunk in grouped:
            (selling_dealer, selling_dealerid,
             selling_location, selling_locationid,
             to_email, cc_email) = keys

            # Summary for all periods (always 3 rows)
            summary = (
                chunk.groupby("Period", as_index=False)
                .agg({
                    "No of Parts": "sum",
                    "Rejected Order Value (in Rs)": "sum",
                    "Parts Still in Stock (in Rs)": "sum",
                    "Total Rejection": "sum"
                })
                .sort_values("Period")
            )
            summary = summary.set_index("Period").reindex(period_order, fill_value=0).reset_index()

            # 6 months value for subject/body
            last6_value = float(summary.loc[summary["Period"] == "6 Months", "Rejected Order Value (in Rs)"].sum())

            # Send logic: only if any part has positive rejection in 30 Days
            chunk_30 = chunk[chunk["Period"] == "30 Days"]
            send_this = False
            if not chunk_30.empty:
                per_part_30 = (
                    chunk_30.groupby("Partnumber", as_index=False)["Rejected Order Value (in Rs)"].sum()
                )
                send_this = (per_part_30["Rejected Order Value (in Rs)"].fillna(0) > 0).any()

            if not send_this:
                print(f"Skipped email (30 Days per-part value all zero): {selling_dealer} - {selling_location}")
                continue

            # Brand code for subject (optional)
            if "Brand" in chunk.columns and not chunk.empty and pd.notna(chunk["Brand"].iloc[0]):
                brand_full = str(chunk["Brand"].iloc[0]).strip()
            else:
                brand_full = ""
            brand3 = brand_full[:3].upper() if brand_full else ""

            # Subject
            subject = f"Rejection Summary of ₹{inr_format(last6_value)} at {brand3}_{selling_dealer}_{selling_location}"

            # Body
            html_body = build_html_body(selling_dealer, selling_location, summary, last6_value)

            # Excel attachment with 3 sheets
            excel_buffer = io.BytesIO()
            with pd.ExcelWriter(excel_buffer, engine="xlsxwriter") as writer:
                for p in period_order:
                    dfp = chunk[chunk["Period"] == p].copy()

                    # numeric safety
                    for col in ["MRP", "Discount", "Discounted Price per qty", "Qty Required", "Qty Confirmed", "POQty"]:
                        if col in dfp.columns:
                            dfp[col] = pd.to_numeric(dfp[col], errors="coerce")

                    # ensure headers exist
                    for m in (c for c in EXCEL_COL_ORDER if c not in dfp.columns):
                        dfp[m] = ""
                    blank_columns = ["Still_in_stock (QTY)", "Parts Still in Stock (in Rs)"]
                    for col in blank_columns:
                        if col in dfp.columns:
                            dfp[col] = dfp[col].replace({0: ""}).fillna("")
                        

                    dfp = dfp[EXCEL_COL_ORDER] if not dfp.empty else pd.DataFrame(columns=EXCEL_COL_ORDER)
                    dfp.to_excel(writer, index=False, sheet_name=p)
                    

            # Recipients
            to_list = split_email_list(to_email)
            cc_raw = cc_email if (isinstance(cc_email, str) and cc_email.strip()) else DEFAULT_CC
            cc_list = split_email_list(cc_raw)
            # if not to_list:
            #     to_list = cc_list.copy()
            # if not to_list:
            #     print(f"Skipped email (no valid recipients): {selling_dealer} - {selling_location}")
            #     continue

            # Assemble message
            msg = MIMEMultipart()
            msg["From"] = email_user
            msg["To"] = 'scsitin'
            msg["Cc"] = ''
            # msg["To"] = ", ".join(to_list)
            # msg["Cc"] = ", ".join(cc_list)
            msg["Subject"] = subject
            msg.attach(MIMEText(html_body, "html"))

            # Attachment
            attachment = MIMEBase("application", "octet-stream")
            attachment.set_payload(excel_buffer.getvalue())
            encoders.encode_base64(attachment)
            safe_dealer = str(selling_dealer).replace("/", "-").replace("\\", "-")
            safe_loc = str(selling_location).replace("/", "-").replace("\\", "-")
            attachment.add_header(
                "Content-Disposition",
                f'attachment; filename="{safe_dealer}_{safe_loc}_Rejection.xlsx"'
            )
            msg.attach(attachment)

            # Send
            try:
                with smtplib.SMTP(smtp_server, smtp_port, timeout=60) as server:
                    server.starttls()
                    if email_user:
                        server.login(email_user, email_password)
                    server.sendmail(
                        from_addr=email_user or to_list[0],
                        to_addrs=to_list + cc_list,
                        msg=msg.as_string()
                    )
                print(f"Email sent to {selling_dealer} - {selling_location}")
            except Exception as mail_ex:
                print(f"Email sending failed for {selling_dealer} - {selling_location}: {mail_ex}", file=sys.stderr)

    except Exception as ex:
        print(f"Runtime error: {ex}", file=sys.stderr)
    finally:
        try:
            if conn is not None:
                conn.close()
        except:
            pass

if __name__ == "__main__":
    main()


C:\Users\VIRAT RAI\AppData\Local\Temp\ipykernel_30368\496551387.py:1301: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  chunk.groupby("Period", as_index=False)


Email sent to Khanna Hyundai Kanpur - Panki
